In [ ]:
# Using tensorflow python 37 Environment

In [53]:
import logging
from typing import Optional, Tuple

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

from PyEMD.splines import akima, cubic_spline_3pts
from PyEMD.utils import get_timeline

import pylab as plt
import warnings
import time
from pandas import concat
from pandas import DataFrame
from numpy import array

In [54]:
from math import sqrt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation, Dropout, Flatten, Conv1D, MaxPooling1D, ConvLSTM2D
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")

In [ ]:
# 01. Upload Data

In [55]:
df_Gas = pd.read_csv('Gas_close.csv')
df_Gas.set_index(pd.DatetimeIndex(df_Gas['Date']), inplace=True)
df_Gas.drop(['Date'],axis=1,inplace=True)
df_Gas.head()

,Close
Date,
2000-01-04,24.39
2000-01-05,23.73
2000-01-06,23.62
2000-01-07,23.09
2000-01-10,23.73


In [ ]:
# 02. Knowing How Much IMF in each Date

In [ ]:
# Upload Data Jumlah Kol

In [56]:
df_Kol = pd.read_csv('Jumlah_IMF_EEMD.csv')
df_Kol.set_index(pd.DatetimeIndex(df_Kol['Date']), inplace=True)
df_Kol.drop(['Date'],axis=1,inplace=True)
df_Kol.head()

,Jum_Kol
Date,
2018-11-27,10
2018-11-28,11
2018-11-29,10
2018-11-30,11
2018-12-03,11


In [ ]:
# 03. Univariate LSTM

In [ ]:
# Prediksi data IMF 1 semua

In [57]:
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [58]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [59]:
def series_to_supervised(data, n_in, n_out=30):
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg.values

In [60]:
# 5. Difference Dataset
def difference(data, interval):
    return [data[i] - data[i - interval] for i in range(interval, len(data))]

In [61]:
# Untuk Horizon lebih dari satu
def split_sequence2(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [8]:
# Testing 2 Hor :

In [62]:
lim = 900
iIMF = 1 # Letak IMF yang diambil
n_test = 30 # horizon = 2
pred_iIMF = []

In [63]:
pred_all = []
n_features = 1 # klo horizon ini tetap satu saja

In [64]:
# Parameter and Define LSTM Model 
n_steps = 50 # Dari 50, 10 lebih baik
n_input = 50
n_steps_out = 30
n_nodes = 50 # Dari 70, kayaknya bagusnya 10
n_epochs = 100 # Dari 50
n_batch = 100
n_diff = 0 # Diff dirubah ke 1

In [ ]:
# Ini Proses Sekaligus semua IMF

In [65]:
start = time.time()
lim = 900
n_test = 30 # horizon = 2
# cfg = [24, 500, 100, 100] # [24, 500, 100, 100]
pred = []

for i in range (1, 121):
    if i % 30 != 0:
        continue
    df_coba2 = pd.read_csv(f'EEMD_-{lim-i}.csv')
    df_coba2.set_index(pd.DatetimeIndex(df_coba2['Date']), inplace=True)
    df_coba2.drop(['Date'],axis=1,inplace=True)
    bar, kol = df_coba2.shape
    all_pred = np.zeros(30) # dua elemen karena horizon 2
    
    for j in range (0, kol):
        imfs_x = df_coba2[f'{j+1}'].values
        train, test = train_test_split(imfs_x, n_test)
        if n_diff > 0:
            train = difference(train, n_diff)
        
        data = series_to_supervised(train, n_input)
        train_x, train_y = data[:, :-30], data[:, -30:]
        train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], 1))
        
        model = Sequential()
        model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, 1)))
        model.add(Dense(n_nodes, activation='relu'))
        model.add(Dense(30))
        model.compile(loss='mse', optimizer='adam')
        
        model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
                               
        x_input = train[-n_steps:].reshape((1, n_steps))
        yhat = model.predict(x_input, verbose=0)
        
        for k in range(0, n_steps_out):
            all_pred[k] += list(yhat[0])[k]
                   
    for l in range(0, n_steps_out):
        pred.append(all_pred[l])
    print(' > proses ke %i' % i)

print('it took: ', (time.time()-start)*1000)
pred  

 > proses ke 30
 > proses ke 60
 > proses ke 90
 > proses ke 120
it took:  9394590.631008148


[-88.80351603031158,
 14.858022451400757,
 107.9429243505001,
 -130.410226136446,
 47.99008631706238,
 77.87670770287514,
 93.50774395465851,
 106.72164857387543,
 53.0276643037796,
 5.643468976020813,
 75.06029972434044,
 92.1914287507534,
 78.3440934419632,
 -84.34877854585648,
 26.63776820898056,
 37.046310901641846,
 101.43027806282043,
 145.69525983929634,
 45.23157811164856,
 -20.24144172668457,
 117.24974718689919,
 -18.990518152713776,
 38.957644551992416,
 31.97596150636673,
 28.481517642736435,
 51.95405727624893,
 15.95708253979683,
 166.2665628194809,
 114.44882154464722,
 145.5524840950966,
 61.41214461624622,
 60.77880221605301,
 60.88410210609436,
 61.44996263086796,
 61.311752900481224,
 62.19717100262642,
 60.81135085225105,
 61.16778714954853,
 60.87514674663544,
 61.347888737916946,
 61.86378040909767,
 60.5234339684248,
 61.23839683830738,
 61.61733515560627,
 60.49123264849186,
 60.20795100927353,
 60.72142006456852,
 60.842030957341194,
 60.551159113645554,
 61.50

In [50]:
'''lim = 920
i = 30
j =0

df_coba2 = pd.read_csv(f'EMD_-{lim-i}.csv')
df_coba2.set_index(pd.DatetimeIndex(df_coba2['Date']), inplace=True)
df_coba2.drop(['Date'],axis=1,inplace=True)
bar, kol = df_coba2.shape
all_pred = np.zeros(30) # dua elemen karena horizon 2

imfs_x = df_coba2[f'{j+1}'].values
train, test = train_test_split(imfs_x, n_test)
if n_diff > 0:
    train = difference(train, n_diff)
        
data = series_to_supervised(train, n_input)
train_x, train_y = data[:, :-30], data[:, -30:]
train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], 1))
        
model = Sequential()
model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, 1)))
model.add(Dense(n_nodes, activation='relu'))
model.add(Dense(30))
model.compile(loss='mse', optimizer='adam')
model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)'''

In [51]:
x_input = train[-n_steps:].reshape((1, n_steps))
# x_input = array(history[-n_input:]).reshape(1, n_input)
# forecast
yhat = model.predict(x_input, verbose=0)

In [52]:
yhat

array([[-0.3900623 , -0.06710275,  0.07675873,  0.06452085,  0.03760254,
         0.0266528 , -0.0317301 , -0.05068859, -0.00640512,  0.03515866,
        -0.03019301, -0.01423518,  0.06532157,  0.03297314, -0.03280794,
        -0.01462862,  0.01442577,  0.02967494, -0.0138556 ,  0.0058794 ,
        -0.00891548, -0.0031378 ,  0.00820029,  0.00740119, -0.02397521,
         0.00289612, -0.0037009 ,  0.03252812, -0.00307405,  0.02634454]],
      dtype=float32)

In [ ]:
# Ini Yang sekaligus IMF

In [66]:
# INGAT NEXT adalah
# Pengambilan s/d 121 - 151 ; len harusnya 5, 10 atau 15; total sampai 246 - 251, s/d Proses 50
# Proses 24 ke 30 
for k in range(0, len(pred)):
    pred_all.append(pred[k])
len(pred_all)

120

In [15]:
# Simpan File tiap 50
lim = 900
i = 120
df_coba3 = pd.DataFrame(df_Gas.index[-900:-780].values, columns=['Date'])
df_coba3['pred'] = pred_all
df_coba3.set_index(pd.DatetimeIndex(df_coba3['Date']), inplace=True)
df_coba3.drop(['Date'],axis=1,inplace=True)
df_coba3.to_csv(f'Pred_LSTM30_EEMD-{lim}-{lim-i}.csv',index=True)
df_coba3.to_excel(f'Pred_LSTM30_EEMD-{lim}-{lim-i}.xlsx',sheet_name='Sheet1',index=True)

In [ ]:
# 04. Hitung Error Measurement

In [68]:
def timeseries_evaluation_metrics_func(y_true, y_pred):    
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true, y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')